In [ ]:
# Current directory
import os
os.chdir('F:/One/ACP')

### Importing Data

In [ ]:
import numpy as np
import pandas as pd


# train
train_features = pd.read_csv("features_label/train.csv", index_col=False, header=None)
train_labels = pd.read_csv("features_label/train_label.csv", index_col=False, header=None)
# test
test_features = pd.read_csv("features_label/test.csv", index_col=False, header=None)
test_labels = pd.read_csv("features_label/test_label.csv", index_col=False, header=None)

print('train_features:', train_features.shape)
print('train_labels', train_labels.shape)
print('test_features', test_features.shape)
print('test_labels', test_labels.shape)


train_features = np.array(train_features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
test_labels = np.array(test_labels)


### Loading Environment

In [ ]:
import statistics
import os,sys,math
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from keras.optimizers import adam_v2
from keras.activations import sigmoid
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential, Model
from keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix
from keras.layers import multiply, Add, Permute
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation
from keras import initializers, regularizers, constraints
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import Input, Dense, Layer, Reshape, Flatten
from keras.layers import Dropout, Lambda, Concatenate, Multiply
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D


def get_shuffle(data,label):    
    #shuffle data
    index = [i for i in range(len(label))]
    np.random.shuffle(index)
    data = data[index]
    label = label[index]
    return data,label 


def scale_mean_var(input_arr,axis=0):
    #from sklearn import preprocessing
    #input_arr= preprocessing.scale(input_arr.astype('float'))
    mean_ = np.mean(input_arr,axis=0)
    scale_ = np.std(input_arr,axis=0)
    # 减均值 
    output_arr= input_arr- mean_
    # 判断均值是否接近0
    mean_1 = output_arr.mean(axis=0)
    if not np.allclose(mean_1, 0):
        output_arr -= mean_1
    # 将标准差为0元素的置1
    #scale_ = _handle_zeros_in_scale(scale_, copy=False)
    scale_[scale_ == 0.0] = 1.0
    # 除以标准差
    output_arr /=scale_
    # 再次判断均值是否为0
    mean_2 = output_arr .mean(axis=0)
    if not np.allclose(mean_2, 0):
        output_arr  -= mean_2

    return output_arr


########################################################### Def Cbam
def channel_attention(input_feature, ratio=8):
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]
	shared_layer_one = Dense(channel//ratio,
							 kernel_initializer='he_normal',
							 activation = 'relu',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('hard_sigmoid')(cbam_feature)
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	return multiply([input_feature, cbam_feature])


def spatial_attention(input_feature):
	kernel_size = 7
	if K.image_data_format() == "channels_first":
		channel = input_feature.shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature.shape[-1]
		cbam_feature = input_feature
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool.shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool.shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat.shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					activation = 'hard_sigmoid',
					strides=1,
					padding='same',
					kernel_initializer='he_normal',
					use_bias=False)(concat)
	assert cbam_feature.shape[-1] == 1
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	return multiply([input_feature, cbam_feature])


def cbam_block(cbam_feature,ratio=8):
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature, )
	return cbam_feature


############################################## Def discriminator and generator
def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


def build_discriminator():
    img = Input(shape=(1,input_dimwx,1))
    x = Conv2D(filters=64, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    x = Conv2D(filters=32, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    """
    NOTE: Capsule architecture starts from here.
    """
    ##### primarycaps coming first ##### 
    x = Conv2D(filters=32, kernel_size=(1,3), strides=2, padding='valid', name='primarycap_conv2')(x)    
    [aa,bb,cc,dd] = x.shape
    numx = int(cc)
    x = Reshape(target_shape=[-1, numx], name='primarycap_reshape')(x)
    x = Lambda(squash, name='primarycap_squash')(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    ##### digitcaps are here ##### 
    x = Flatten()(x)
    uhat = Dense(128, kernel_initializer='he_normal', bias_initializer='zeros', name='uhat_digitcaps')(x)
    c = Activation('softmax', name='softmax_digitcaps1')(uhat) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(128)(c) # compute s_j
    x = Multiply()([uhat, c])
    """
    NOTE: Squashing the capsule outputs creates severe blurry artifacts, thus we replace it with Leaky ReLu.
    """
    s_j = LeakyReLU()(x)
    ##### we will repeat the routing part 2 more times (num_routing=3) to unfold the loop
    c = Activation('softmax', name='softmax_digitcaps2')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(128)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)

    c = Activation('softmax', name='softmax_digitcaps3')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(128)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)
    
    c = Activation('softmax', name='softmax_digitcaps4')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(128)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)
    # ##### preparition for cbam_block
    s_j = Reshape((-1,128,1))(s_j)
    inputs = s_j
    residual = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', name='convxxx')(inputs)
    residual = BatchNormalization(momentum=0.8)(residual)
    
    cbam = cbam_block(residual)
    # cbam = channel_attention(residual)
    # cbam = spatial_attention(residual)
    
    cbam = Reshape((-1,))(cbam)
    pred = Dense(2, activation='sigmoid')(cbam)
    
    # cbam = Reshape((-1,))(s_j)
    # pred = Dense(2, activation='sigmoid')(cbam)
    
    
    return Model(img, pred)


# generator structure
def build_generator():
    """
    Generator follows the DCGAN architecture and creates generated image representations through learning.
    """
    noise_shape =(input_dimwx,)
    x_noise = Input(shape=noise_shape)
    # we apply different kernel sizes in order to match the original image size
    x = Dense(64 * 1 * input_dimwx, activation="relu")(x_noise)
    x = Reshape((1, input_dimwx, 64))(x)
    x = BatchNormalization(momentum=0.2)(x)
    x = UpSampling2D()(x)
    [aa1,bb1,cc1,dd1] = x.shape
    numx1 = int(cc1//4)
    x = Conv2D(32, kernel_size=(2,numx1), padding="valid")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.2)(x)
    [aa2,bb2,cc2,dd2] = x.shape
    #### x = UpSampling2D()(x)
    numx2 = int(1+cc2-input_dimwx)
    x = Conv2D(16, kernel_size=(1,numx2), padding="valid")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.2)(x)
    x = Conv2D(1, kernel_size=3, padding="same")(x)
    gen_out = Activation("tanh")(x)
        
    return Model(x_noise, gen_out)


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)


def to_categorical(y, nb_classes=None):
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y



### Training and prediction

#### Independent testing

In [ ]:
# Create a directory to save the model
model_save_dir = 'save_models/CapsuleGAN/Independence'
os.makedirs(model_save_dir, exist_ok=True)

In [ ]:
# train_set = scale(X_new)
[sample_num, input_dimwx] = np.shape(train_features)
X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels

ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]

mean_recall = np.linspace(0, 1, 100)
all_precision = []
base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
interp_tpr_collection = []

losses = []  # 用于存储每个fold的损失值
for i in range(10):
    # Loading the model
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_v2.Adam(0.001), metrics=['accuracy'])

    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=adam_v2.Adam(0.001), metrics=['accuracy'])

    # feeding noise to generator
    z = Input(shape=(input_dimwx,))
    img = generator(z)
    # for the combined model we will only train the generator
    discriminator.trainable = False
    # try to discriminate generated images
    valid = discriminator(img)
    # the combined model (stacked generator and discriminator) takes
    # noise as input => generates images => determines validity 
    combined = Model(z, valid)
    combined.compile(loss='binary_crossentropy', optimizer=adam_v2.Adam(0.001), metrics=['accuracy'])
    
    # dataset
    y_train_whole = to_categorical(y_train)
    cv_clf = combined
    hist = cv_clf.fit(X_train, y_train_whole, batch_size=64, epochs=90)

    losses.append(hist.history['loss'])  # 记录每个epoch的损失值

    y_score = cv_clf.predict(X_test)
    y_class = categorical_probas_to_classes(y_score)
    TP, FP, FN, TN = confusion_matrix(y_test, y_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
    interp_tpr = np.interp(base_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tpr_collection.append(interp_tpr)
    auc_roc = auc(fpr, tpr)
    AUC_collecton.append(auc_roc)
    # PR curve
    precision, recall, _ = precision_recall_curve(y_test, y_score[:, 1])
    average_precision = average_precision_score(y_test, y_score[:, 1])
    recall = np.flipud(recall)
    precision = np.flipud(precision)

    mean_precision = np.interp(mean_recall, recall, precision)
    all_precision.append(mean_precision)
    AP.append(average_precision)

# 在所有交叉验证循环结束后,计算TPR的均值
mean_tpr = np.mean(interp_tpr_collection, axis=0)
mean_tpr[-1] = 1.0

# Calculate the mean precision
mean_precision = np.mean(all_precision, axis=0)

# Output
results = [
    f"ACC: {round(statistics.mean(ACC_collecton), 3)} ± {round(statistics.stdev(ACC_collecton), 3)}",
    f"BACC: {round(statistics.mean(BACC_collecton), 3)} ± {round(statistics.stdev(BACC_collecton), 3)}",
    f"Sn: {round(statistics.mean(Sn_collecton), 3)} ± {round(statistics.stdev(Sn_collecton), 3)}",
    f"Sp: {round(statistics.mean(Sp_collecton), 3)} ± {round(statistics.stdev(Sp_collecton), 3)}",
    f"MCC: {round(statistics.mean(MCC_collecton), 3)} ± {round(statistics.stdev(MCC_collecton), 3)}",
    f"AUC: {round(statistics.mean(AUC_collecton), 3)} ± {round(statistics.stdev(AUC_collecton), 3)}",
    f"AP: {round(statistics.mean(AP), 3)} ± {round(statistics.stdev(AP), 3)}"
]

for result in results:
    print(result)

# Append the results to the file
with open('result/results_CapsuleGAN.txt', 'a') as file:
    file.write("----------------------------------------\n")
    for result in results:
        file.write(result + '\n')



In [ ]:
# 绘制损失函数曲线
from matplotlib.ticker import FuncFormatter

n = 0
loss = losses[n]
plt.figure()

plt.plot(loss)
# for i, loss in enumerate(losses):
#     plt.plot(loss, label=f'Fold {i+1}')

formatter = FuncFormatter(lambda x, pos: f'{x:.2f}')
plt.gca().yaxis.set_major_formatter(formatter)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.tight_layout()
# plt.savefig('loss_curve.png', dpi=300)
plt.show()

In [ ]:
# 绘制ROC曲线
plt.figure()
lw = 2
plt.plot(base_fpr, mean_tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % np.mean(AUC_collecton))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Independence test')
plt.legend(loc="lower right")
plt.tight_layout()
# plt.savefig('loss_curve.png', dpi=300)
plt.show()

### end